<span style="color:#FF7A00; font-size:30px; font-weight:bold;">
BDP Term Project (1조)
</span>

<span style="font-size:20px; font-weight:bold;">
encar.com에서 국산 중고차 매물별 보험이력 불러오기
</span>

<span style="font-size:17px; font-weight:bold;">
Driver Load
</span>

In [3]:
import csv
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ChromeDriver 경로 지정
driver_path = "./chromedriver-win64/chromedriver.exe" 
# Chrome WebDriver 실행
driver = webdriver.Chrome(executable_path=driver_path)

print("작업 완료")

작업 완료


<span style="font-size:17px; font-weight:bold;">
Crawling
</span>

In [5]:
def fetch_vehicle_ids(driver, base_url, pages=1):
    # 매물 ID 불러오기
    vehicle_ids = set()
    for page in range(1, pages + 1):
        driver.get(base_url.format(page=page))
        # 요소 팝업까지 대기
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//div[@data-impression]"))
            )
        except Exception:
            print(f"페이지 {page} 로드 실패. 다음 페이지로 이동합니다.")
            continue
        elements = driver.find_elements(By.XPATH, "//div[@data-impression]")
        vehicle_ids.update(
            e.get_attribute("data-impression").split("|")[0] for e in elements if e.get_attribute("data-impression")
        )
    return list(vehicle_ids)

def fetch_insurance_history(driver, vehicle_ids, output_file):
    # ID에 부합하는 보험이력 정보 저장
    '''
    ID : Vehicle ID (매물 ID)
    N : Car Number (자동차 번호)
    NC : Number Change History (번호 변경 이력)
    OC : Owner Change History (소유자 변경 이력)
    TL : Total Loss History (전손 사고 이력)
    FD : Flood Damage History (침수 사고 이력)
    TF : Theft History (도난 사고 이력)
    CM : Insurance Claim of My Car (내 차 피해 이력)
    CO : Insurance Claim of Other Car (타 차 가해 이력)
    '''
    fieldnames = ["ID", "N", "NC", "OC","TL", "FD", "TF","CM", "CO"]
    data = []
    for vid in vehicle_ids:
        try:
            driver.get(f"https://fem.encar.com/cars/report/accident/{vid}")
            details = {
                "ID": vid,
                "N": WebDriverWait(driver, 1).until(
                    EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/dl/dd[1]"))
                ).text,
                "NC": driver.find_element(By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[1]").text,
                "OC": driver.find_element(By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[2]").text,
                "TL": driver.find_element(By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[1]").text,
                "FD": driver.find_element(By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[2]").text,
                "TF": driver.find_element(By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[3]").text,
                "CM": driver.find_element(By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[5]/span").text,
                "CO": driver.find_element(By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[6]/span").text
            }
            data.append(details)
        except Exception:
            print(f"ID '{vid}': 판매자가 보험이력을 공개하지 않았습니다.")
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

def merge_csv_files(files, output_file):
    # csv파일 최종적으로 하나로 병합
    combined_df = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)
    combined_df.to_csv(output_file, index=False, encoding="utf-8")
    print(f"'{output_file}'파일에 병합되어 저장되었습니다.")

def main():
    # 국산차 브랜드별 필터링 리스트
    urls = {
        "HYUNDAI": "https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.%ED%98%84%EB%8C%80.))%22%2C%22title%22%3A%22%ED%98%84%EB%8C%80%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D",
        "GENESIS" : "https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.%EC%A0%9C%EB%84%A4%EC%8B%9C%EC%8A%A4.))%22%2C%22title%22%3A%22%EC%A0%9C%EB%84%A4%EC%8B%9C%EC%8A%A4%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D",
        "KIA": "https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.%EA%B8%B0%EC%95%84.))%22%2C%22title%22%3A%22%EA%B8%B0%EC%95%84%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D",
        "CHEVROLET": "https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.%EC%89%90%EB%B3%B4%EB%A0%88(GM%EB%8C%80%EC%9A%B0_).))%22%2C%22title%22%3A%22%EC%89%90%EB%B3%B4%EB%A0%88(GM%EB%8C%80%EC%9A%B0)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D",
        "RENAULT": "https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.%EB%A5%B4%EB%85%B8%EC%BD%94%EB%A6%AC%EC%95%84(%EC%82%BC%EC%84%B1_).))%22%2C%22title%22%3A%22%EB%A5%B4%EB%85%B8%EC%BD%94%EB%A6%AC%EC%95%84(%EC%82%BC%EC%84%B1)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D",
        "KGMOBILITY": "https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.KG%EB%AA%A8%EB%B9%8C%EB%A6%AC%ED%8B%B0(%EC%8C%8D%EC%9A%A9_).))%22%2C%22title%22%3A%22KG%EB%AA%A8%EB%B9%8C%EB%A6%AC%ED%8B%B0(%EC%8C%8D%EC%9A%A9)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D"
    }

    # ChromeDriver 경로 지정
    driver_path = "./chromedriver-win64/chromedriver.exe" 
    
    # Chrome WebDriver 실행
    driver = webdriver.Chrome(executable_path=driver_path)

    try:
        csv_files = []
        for brand, url in urls.items():
            print(f"[{brand}] 탐색 중...")
            ids = fetch_vehicle_ids(driver, url)
            output_file = f"insurance_{brand}.csv"
            fetch_insurance_history(driver, ids, output_file)
            csv_files.append(output_file)
            print(f"총 매물{len(ids)}개 탐색 완료")
            print(f"===========================================================")
        merge_csv_files(csv_files, "insurance_all.csv")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

[HYUNDAI] 탐색 중...
ID '38594498': 판매자가 보험이력을 공개하지 않았습니다.
ID '38422413': 판매자가 보험이력을 공개하지 않았습니다.
ID '38305155': 판매자가 보험이력을 공개하지 않았습니다.
ID '37304198': 판매자가 보험이력을 공개하지 않았습니다.
ID '37999816': 판매자가 보험이력을 공개하지 않았습니다.
총 매물249개 탐색 완료
[GENESIS] 탐색 중...
ID '38002836': 판매자가 보험이력을 공개하지 않았습니다.
ID '38350220': 판매자가 보험이력을 공개하지 않았습니다.
ID '38511778': 판매자가 보험이력을 공개하지 않았습니다.
ID '38582256': 판매자가 보험이력을 공개하지 않았습니다.
ID '38046836': 판매자가 보험이력을 공개하지 않았습니다.
총 매물220개 탐색 완료
[KIA] 탐색 중...
ID '37724384': 판매자가 보험이력을 공개하지 않았습니다.
ID '38636700': 판매자가 보험이력을 공개하지 않았습니다.
ID '38296232': 판매자가 보험이력을 공개하지 않았습니다.
총 매물245개 탐색 완료
[CHEVROLET] 탐색 중...
ID '38225186': 판매자가 보험이력을 공개하지 않았습니다.
ID '37954010': 판매자가 보험이력을 공개하지 않았습니다.
ID '38227512': 판매자가 보험이력을 공개하지 않았습니다.
ID '38455220': 판매자가 보험이력을 공개하지 않았습니다.
총 매물224개 탐색 완료
[RENAULT] 탐색 중...
ID '36800906': 판매자가 보험이력을 공개하지 않았습니다.
ID '37941557': 판매자가 보험이력을 공개하지 않았습니다.
ID '38131302': 판매자가 보험이력을 공개하지 않았습니다.
총 매물231개 탐색 완료
[KGMOBILITY] 탐색 중...
ID '38354711': 판매자가 보험이력을 공개하지 않았습니다.
ID '38582947': 판매자